In [3]:
import pandas as pd

In [4]:
import pandas as pd
import numpy as np

#NLTK
import nltk
from nltk import word_tokenize, WordPunctTokenizer, regexp_tokenize
from nltk import word_tokenize, WordPunctTokenizer, regexp_tokenize

#Plotting 
import matplotlib.pyplot as plt


#Keras
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding


2023-05-04 01:43:42.636767: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-04 01:43:42.684508: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-04 01:43:42.684960: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 01:43:43.503379: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
def score(category):
    dict = {'18-22F' : 0.85556179, '23-45F' : 0.87431270, '46+F' : 0.83906224, '46+M' : 0.83220058, '23-45M' : 0.83609059, '18-22M' : 0.84184846}
    return dict[category]

def label(genders, ages, labels):
    yes = 0
    no = 0
    for i in range(0, 6):
        category = ages[i] + genders[i]
        if labels[i] == 'YES':
            yes += score(category)
        else:
            no += score(category)
    if yes > no:
        return 'YES'
    else:
        return 'NO'


In [7]:
train_yes_es = pd.read_json('training/EXIST2023_training.json')
train_yes_es=train_yes_es.transpose()
train_yes_es=train_yes_es.loc[train_yes_es['lang'].apply(lambda x:x=='es')]
train_yes_es=train_yes_es.loc[train_yes_es['labels_task1'].apply(lambda x:x.count('YES')>3)]
train_yes_es['label1']='YES'

#NO

train_no_es = pd.read_json('training/EXIST2023_training.json')
train_no_es=train_no_es.transpose()
train_no_es=train_no_es.loc[train_no_es['lang'].apply(lambda x:x=='es')]
train_no_es=train_no_es.loc[train_no_es['labels_task1'].apply(lambda x:x.count('NO')>3)]
train_no_es['label1']='NO'


train_amb = pd.read_json('training/EXIST2023_training.json')
train_amb=train_amb.transpose()
train_amb=train_amb.loc[train_amb['lang'].apply(lambda x:x=='es')]
train_amb=train_amb.loc[train_amb['labels_task1'].apply(lambda x:x.count('YES')==3)]
for i, sample in enumerate(train_amb.itertuples()):
    truelabel = label(sample[6], sample[7], sample[8])
    train_amb.at[sample[0], 'label1'] = truelabel

In [8]:
validation_yes_es = pd.read_json('dev/EXIST2023_dev.json')
validation_yes_es=validation_yes_es.transpose()
validation_yes_es=validation_yes_es.loc[validation_yes_es['lang'].apply(lambda x:x=='es')]
validation_yes_es=validation_yes_es.loc[validation_yes_es['labels_task1'].apply(lambda x:x.count('YES')>3)]
validation_yes_es['label1']='YES'

#NO

validation_no_es = pd.read_json('dev/EXIST2023_dev.json')
validation_no_es=validation_no_es.transpose()
validation_no_es=validation_no_es.loc[validation_no_es['lang'].apply(lambda x:x=='es')]
validation_no_es=validation_no_es.loc[validation_no_es['labels_task1'].apply(lambda x:x.count('NO')>3)]
validation_no_es['label1']='NO'

In [9]:
frames = [validation_yes_es, validation_no_es]
validation = pd.concat(frames)
validation.reset_index(drop=True)

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
0,300002,es,@anacaotica88 @MordorLivin No me acuerdo de lo...,6,"[Annotator_731, Annotator_732, Annotator_315, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, NO, YES, YES, YES]","[JUDGEMENTAL, REPORTED, -, JUDGEMENTAL, JUDGEM...","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",DEV_ES,YES
1,300004,es,Also mientras les decia eso la señalaba y deci...,6,"[Annotator_259, Annotator_739, Annotator_291, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, YES, YES, YES]","[-, REPORTED, REPORTED, REPORTED, JUDGEMENTAL,...","[[-], [SEXUAL-VIOLENCE], [SEXUAL-VIOLENCE], [S...",DEV_ES,YES
2,300007,es,@DavidGR18 @pppbernat @abc_es @agarzon @IreneM...,6,"[Annotator_731, Annotator_732, Annotator_315, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, NO, YES, YES, YES]","[DIRECT, DIRECT, -, DIRECT, JUDGEMENTAL, REPOR...","[[IDEOLOGICAL-INEQUALITY, SEXUAL-VIOLENCE, MIS...",DEV_ES,YES
3,300008,es,@DavidArranzVox @AnabelAlonso_of Uyyy a q huel...,6,"[Annotator_742, Annotator_743, Annotator_195, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, NO, YES, YES, YES]","[JUDGEMENTAL, JUDGEMENTAL, -, REPORTED, JUDGEM...","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",DEV_ES,YES
4,300010,es,@kokreto84 @Play87834898 @venusoncrack Me gust...,6,"[Annotator_744, Annotator_745, Annotator_746, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, YES, YES, YES, YES]","[DIRECT, REPORTED, DIRECT, JUDGEMENTAL, DIRECT...","[[MISOGYNY-NON-SEXUAL-VIOLENCE], [STEREOTYPING...",DEV_ES,YES
...,...,...,...,...,...,...,...,...,...,...,...,...
485,300534,es,El Gobernador @samuel_garcias se da el lujo d...,6,"[Annotator_726, Annotator_727, Annotator_357, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, NO, NO, NO]","[-, REPORTED, JUDGEMENTAL, -, -, -]","[[-], [IDEOLOGICAL-INEQUALITY], [STEREOTYPING-...",DEV_ES,NO
486,300538,es,Violencia simbólica también es un Presidente q...,6,"[Annotator_750, Annotator_751, Annotator_189, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, NO, NO, NO]","[-, -, -, -, -, -]","[[-], [-], [-], [-], [-], [-]]",DEV_ES,NO
487,300539,es,En el #podcast Un día de libros 67: Violencia ...,6,"[Annotator_754, Annotator_320, Annotator_123, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, NO, NO, NO]","[-, REPORTED, -, -, -, -]","[[-], [MISOGYNY-NON-SEXUAL-VIOLENCE], [-], [-]...",DEV_ES,NO
488,300540,es,Top story: NBA tiene menos actos de violencia ...,6,"[Annotator_259, Annotator_739, Annotator_291, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, NO, YES, NO]","[-, -, -, -, JUDGEMENTAL, -]","[[-], [-], [-], [-], [MISOGYNY-NON-SEXUAL-VIOL...",DEV_ES,NO


In [10]:
frames = [train_yes_es, train_no_es,train_amb]
train = pd.concat(frames)
train.reset_index(drop=True)

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
0,100001,es,"@TheChiflis Ignora al otro, es un capullo.El p...",6,"[Annotator_1, Annotator_2, Annotator_3, Annota...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, YES, NO, YES, YES, YES]","[REPORTED, JUDGEMENTAL, -, REPORTED, JUDGEMENT...","[[OBJECTIFICATION], [OBJECTIFICATION, SEXUAL-V...",TRAIN_ES,YES
1,100005,es,@novadragon21 @icep4ck @TvDannyZ Entonces como...,6,"[Annotator_19, Annotator_20, Annotator_21, Ann...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, NO, YES, NO, YES, YES]","[REPORTED, -, JUDGEMENTAL, -, JUDGEMENTAL, DIR...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION], [-...",TRAIN_ES,YES
2,100008,es,@BestKabest Esta gringa sigue llorando por el ...,6,"[Annotator_25, Annotator_26, Annotator_27, Ann...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, DIRECT, JUDGEMENTAL, DIRECT, ...","[[IDEOLOGICAL-INEQUALITY], [STEREOTYPING-DOMIN...",TRAIN_ES,YES
3,100028,es,"@ShahidForChange @TeamPelosi Quiet, sexist ^%$...",6,"[Annotator_109, Annotator_110, Annotator_111, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, NO, NO, YES, YES, YES]","[JUDGEMENTAL, -, -, DIRECT, DIRECT, DIRECT]","[[STEREOTYPING-DOMINANCE], [-], [-], [IDEOLOGI...",TRAIN_ES,YES
4,100036,es,@Harassed_girl loca d mierda en k momento,6,"[Annotator_115, Annotator_116, Annotator_117, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, YES, YES, YES, YES, NO]","[DIRECT, DIRECT, DIRECT, DIRECT, DIRECT, -]","[[STEREOTYPING-DOMINANCE, SEXUAL-VIOLENCE], [M...",TRAIN_ES,YES
...,...,...,...,...,...,...,...,...,...,...,...,...
3655,103618,es,SUJETO QUE VIOLÓ A UNA MENOR ES SENTENCIADO A ...,6,"[Annotator_205, Annotator_206, Annotator_207, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, NO, NO, NO, YES, YES]","[REPORTED, -, -, -, REPORTED, REPORTED]","[[SEXUAL-VIOLENCE], [-], [-], [-], [SEXUAL-VIO...",TRAIN_ES,NO
3656,103625,es,- Aaaaaaaa• La Zorra abria la boca y se estamp...,6,"[Annotator_175, Annotator_176, Annotator_177, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, NO, NO, YES, YES, NO]","[DIRECT, -, -, DIRECT, REPORTED, -]","[[OBJECTIFICATION], [-], [-], [OBJECTIFICATION...",TRAIN_ES,NO
3657,103629,es,@strangrbrina ahí te contesto PERO YOCSOY TU A...,6,"[Annotator_343, Annotator_344, Annotator_345, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, YES, NO, NO, YES, NO]","[DIRECT, DIRECT, -, -, DIRECT, -]","[[MISOGYNY-NON-SEXUAL-VIOLENCE], [OBJECTIFICAT...",TRAIN_ES,YES
3658,103634,es,@javierLander23 Que vai a ser vieja eskuela vo...,6,"[Annotator_199, Annotator_200, Annotator_201, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, YES, YES, NO, NO, NO]","[JUDGEMENTAL, REPORTED, DIRECT, -, -, -]","[[IDEOLOGICAL-INEQUALITY], [STEREOTYPING-DOMIN...",TRAIN_ES,YES


In [11]:
import re
hash_regex = re.compile(r"#(\w+)")
hstgs = [] # To store the hashtags so we can exclude them from some parts of the analysis
def hash_repl(match):
    _ = '__HASH_'+match.group(1).upper()
    hstgs.append(_)
    return _

In [12]:
user_regex = re.compile(r"@(\w+)")
usr_names = [] # To store the user names so we can exclude them from some parts of the analysis
def user_repl(match):
    _ = '__user_'+match.group(1).upper()
    usr_names.append(_)
    return _

In [13]:
url_regex = re.compile(r"(http|https|ftp)://[a-zA-Z0-9\./]+")
def url_repl(match):
    return '__URL_'

In [14]:
# Repeating words like hurrrryyyyyy
rpt_regex = re.compile(r"(.)\1{1,}", re.IGNORECASE);
def rpt_repl(match):
    return match.group(1)+match.group(1)


In [15]:
# Spliting by word boundaries
word_bound_regex = re.compile(r"\W+")

# Punctuations
punctuations = \
	[	#('',		['.', ] )	,\
		#('',		[',', ] )	,\
		#('',		['\'', '\"', ] )	,\
		('__PUNC_EXCL',		['!', '¡', ] )	,\
		('__PUNC_QUES',		['?', '¿', ] )	,\
		('__PUNC_ELLP',		['...', '…', ] )	,\
	]

#For punctuation replacement
def punctuations_repl(match):
	text = match.group(0)
	repl = []
	for (key, parr) in punctuations :
		for punc in parr :
			if punc in text:
				repl.append(key)
	if( len(repl)>0 ) :
		return ' '+' '.join(repl)+' '
	else :
		return ' '

In [16]:

from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')

In [17]:
def sb_stem(text, only_first=0):
    text = [word if(word[0:2]=='__') else word.lower() for word in text.split() if ((len(word) >= 3) or (word in ['no','si', 'sí', 'ni']))] #   If we are doing negation analysis, maybe is a better idea to keep the small words (like 'no')
    text = [stemmer.stem(w) if w[0:2]!='__' else w for w in text ]
    
    return(text)

In [18]:
def processAll(text):
    text = re.sub( hash_regex, hash_repl, text )
    #text = re.sub( user_regex, user_repl, text)
    text = re.sub( url_regex, url_repl, text )
    
    text = text.replace('\'','')
    
    text = re.sub( word_bound_regex , punctuations_repl, text )
    text = re.sub( rpt_regex, rpt_repl, text )
    
    text = sb_stem(text)    
    return text

In [19]:
train['processed_tweet'] = train.tweet.apply(processAll)
validation['processed_tweet'] = validation.tweet.apply(processAll)

In [20]:
train.head()

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1,processed_tweet
100001,100001,es,"@TheChiflis Ignora al otro, es un capullo.El p...",6,"[Annotator_1, Annotator_2, Annotator_3, Annota...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, YES, NO, YES, YES, YES]","[REPORTED, JUDGEMENTAL, -, REPORTED, JUDGEMENT...","[[OBJECTIFICATION], [OBJECTIFICATION, SEXUAL-V...",TRAIN_ES,YES,"[thechiflis, ignor, otro, capull, problem, con..."
100005,100005,es,@novadragon21 @icep4ck @TvDannyZ Entonces como...,6,"[Annotator_19, Annotator_20, Annotator_21, Ann...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, NO, YES, NO, YES, YES]","[REPORTED, -, JUDGEMENTAL, -, JUDGEMENTAL, DIR...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION], [-...",TRAIN_ES,YES,"[novadragon21, icep4ck, tvdannyz, entonc, com,..."
100008,100008,es,@BestKabest Esta gringa sigue llorando por el ...,6,"[Annotator_25, Annotator_26, Annotator_27, Ann...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, DIRECT, JUDGEMENTAL, DIRECT, ...","[[IDEOLOGICAL-INEQUALITY], [STEREOTYPING-DOMIN...",TRAIN_ES,YES,"[bestkabest, esta, gring, sig, llor, por, game..."
100028,100028,es,"@ShahidForChange @TeamPelosi Quiet, sexist ^%$...",6,"[Annotator_109, Annotator_110, Annotator_111, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, NO, NO, YES, YES, YES]","[JUDGEMENTAL, -, -, DIRECT, DIRECT, DIRECT]","[[STEREOTYPING-DOMINANCE], [-], [-], [IDEOLOGI...",TRAIN_ES,YES,"[shahidforchang, teampelosi, quiet, sexist, __..."
100036,100036,es,@Harassed_girl loca d mierda en k momento,6,"[Annotator_115, Annotator_116, Annotator_117, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, YES, YES, YES, YES, NO]","[DIRECT, DIRECT, DIRECT, DIRECT, DIRECT, -]","[[STEREOTYPING-DOMINANCE, SEXUAL-VIOLENCE], [M...",TRAIN_ES,YES,"[harassed_girl, loc, mierd, moment]"


In [21]:
train_data = pd.DataFrame()
train_data['tweet'] = train['processed_tweet']
train_data['label'] = train['label1']
train_data.reset_index(inplace=True, drop = True)
train_data

,tweet,label
0,"[thechiflis, ignor, otro, capull, problem, con...",YES
1,"[novadragon21, icep4ck, tvdannyz, entonc, com,...",YES
2,"[bestkabest, esta, gring, sig, llor, por, game...",YES
3,"[shahidforchang, teampelosi, quiet, sexist, __...",YES
4,"[harassed_girl, loc, mierd, moment]",YES
...,...,...
3655,"[sujet, que, viol, una, menor, sentenci, años,...",NO
3656,"[zorr, abri, boc, estamp, contr, intim, del, a...",NO
3657,"[strangrbrin, ahi, contest, per, yocsoy, amig,...",YES
3658,"[javierlander23, que, vai, ser, viej, eskuel, ...",YES


In [22]:
validation_data = pd.DataFrame()
validation_data['tweet'] = validation['processed_tweet']
validation_data['label'] = validation['label1']
validation_data.reset_index(inplace=True, drop = True)
validation_data

,tweet,label
0,"[anacaotica88, mordorlivin, acuerd, los, detal...",YES
1,"[also, mientr, les, deci, eso, señal, deci, qu...",YES
2,"[davidgr18, ppbernat, abc_es, agarzon, irenemo...",YES
3,"[davidarranzvox, anabelalonso_of, uyy, huel, _...",YES
4,"[kokreto84, play87834898, venusoncrack, gust, ...",YES
...,...,...
485,"[gobern, samuel_garci, luj, sub, vide, complet...",NO
486,"[violenci, simbol, tambien, president, que, to...",NO
487,"[__HASH_PODCAST, dia, libr, violenci, simbol, ...",NO
488,"[top, story, nba, tien, men, actos, violenci, ...",NO


In [23]:
print(max(train_data['tweet'].apply(len)))
print(max(validation_data['tweet'].apply(len)))

50
48


In [24]:
from gensim.models import KeyedVectors


In [25]:
vecs = KeyedVectors.load_word2vec_format('SBW-vectors-300-min5.txt', binary=False)

In [26]:
def vectorize_tweet(tweet):
    vec = []
    for word in tweet:
        if word in vecs.key_to_index:
            vec.append(vecs[word])
    return vec

In [27]:
train_data['tweet'] = train_data['tweet'].apply(vectorize_tweet)
validation_data['tweet'] = validation_data['tweet'].apply(vectorize_tweet)

In [28]:
print(max(train_data['tweet'].apply(len)))
print(max(validation_data['tweet'].apply(len)))

42
43


In [29]:
import random

X_train = list(train_data['tweet'])
Y_train = list(train_data['label'])

X_test = list(validation_data['tweet'])
Y_test = list(validation_data['label'])

temp = list(zip(X_train, Y_train))
random.shuffle(temp)
X_train, Y_train = zip(*temp)
X_train = list(X_train)
Y_train = list(Y_train)

In [30]:
import tensorflow as tf

In [31]:
def padto43(x):
    if len(x) < 43:
        for i in range(43 - len(x)):
            x.append([0]*300)
    return x

In [32]:
for i in range(len(X_train)):
    X_train[i] = padto43(X_train[i])

for i in range(len(X_test)):
    X_test[i] = padto43(X_test[i])

In [33]:
X_train_tensor = tf.convert_to_tensor(X_train)
X_test_tensor = tf.convert_to_tensor(X_test)

2023-05-04 01:47:39.421956: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-04 01:47:39.422528: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-05-04 01:47:39.423686: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188856000 exceeds 10% of free system memory.


In [34]:
print(tf.shape(X_train_tensor))
print(tf.shape(X_test_tensor))

tf.Tensor([3660   43  300], shape=(3,), dtype=int32)
tf.Tensor([490  43 300], shape=(3,), dtype=int32)


In [35]:
#convert yes to 1 and no to 0
for i in range(len(Y_train)):
    if Y_train[i] == 'YES':
        Y_train[i] = [1, 0]
    else:
        Y_train[i] = [0, 1]

for i in range(len(Y_test)):
    if Y_test[i] == 'YES':
        Y_test[i] = [1, 0]
    else:
        Y_test[i] = [0, 1]

In [36]:

Y_train_tensor = tf.convert_to_tensor(Y_train)
Y_test_tensor = tf.convert_to_tensor(Y_test)

In [37]:
MLmodel = tf.keras.Sequential([
    tf.keras.layers.LSTM(150),
    tf.keras.layers.Dense(2, activation='softmax')
])
MLmodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
MLmodel.build(input_shape=(None, 43, 300))
MLmodel.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 150)               270600    
                                                                 
 dense (Dense)               (None, 2)                 302       
                                                                 
Total params: 270,902
Trainable params: 270,902
Non-trainable params: 0
_________________________________________________________________


2023-05-04 01:47:43.121363: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-04 01:47:43.122961: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-04 01:47:43.124213: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [38]:
num_epochs = 50
MLmodel.fit(X_train_tensor, Y_train_tensor, epochs=num_epochs, validation_data=(X_test_tensor, Y_test_tensor))

Epoch 1/50


2023-05-04 01:47:43.494651: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-04 01:47:43.496335: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-04 01:47:43.497693: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

115/115 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5046

2023-05-04 01:47:48.425856: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-04 01:47:48.427400: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-04 01:47:48.428669: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

115/115 [==============================] - 6s 32ms/step - loss: 0.6931 - accuracy: 0.5046 - val_loss: 0.6933 - val_accuracy: 0.4918
Epoch 2/50
115/115 [==============================] - 3s 28ms/step - loss: 0.6933 - accuracy: 0.5183 - val_loss: 0.6939 - val_accuracy: 0.4776
Epoch 3/50
115/115 [==============================] - 3s 28ms/step - loss: 0.6876 - accuracy: 0.5552 - val_loss: 0.6655 - val_accuracy: 0.5776
Epoch 4/50
115/115 [==============================] - 3s 28ms/step - loss: 0.6910 - accuracy: 0.5265 - val_loss: 0.6929 - val_accuracy: 0.5000
Epoch 5/50
115/115 [==============================] - 3s 28ms/step - loss: 0.6805 - accuracy: 0.5609 - val_loss: 0.7168 - val_accuracy: 0.4714
Epoch 6/50
115/115 [==============================] - 3s 28ms/step - loss: 0.6633 - accuracy: 0.6005 - val_loss: 0.6505 - val_accuracy: 0.6327
Epoch 7/50
115/115 [==============================] - 3s 28ms/step - loss: 0.6465 - accuracy: 0.6311 - val_loss: 0.6184 - val_accuracy: 0.6551
Epoch 8/50

In [39]:
model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build(input_shape=(None, 43, 300))
model.summary()


2023-05-04 01:51:58.288826: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-04 01:51:58.290678: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-04 01:51:58.292227: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 300)              541200    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 16)                4816      
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_2 (Dense)             (None, 2)                 34        
                                                                 
Total params: 546,050
Trainable params: 546,050
Non-trainable params: 0
________________________________________________

2023-05-04 01:51:58.497821: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-04 01:51:58.499494: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-04 01:51:58.500747: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [40]:
 num_epochs = 50
model.fit(X_train_tensor, Y_train_tensor, epochs=num_epochs, validation_data=(X_test_tensor, Y_test_tensor))


Epoch 1/50


2023-05-04 01:52:02.195792: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-04 01:52:02.197650: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-04 01:52:02.198982: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

115/115 [==============================] - ETA: 0s - loss: 0.6911 - accuracy: 0.5393

2023-05-04 01:52:10.110350: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-04 01:52:10.111868: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-04 01:52:10.113163: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

115/115 [==============================] - 9s 49ms/step - loss: 0.6911 - accuracy: 0.5393 - val_loss: 0.6816 - val_accuracy: 0.5837
Epoch 2/50
115/115 [==============================] - 5s 40ms/step - loss: 0.6658 - accuracy: 0.5926 - val_loss: 0.6514 - val_accuracy: 0.6327
Epoch 3/50
115/115 [==============================] - 5s 41ms/step - loss: 0.6540 - accuracy: 0.6172 - val_loss: 0.6513 - val_accuracy: 0.6020
Epoch 4/50
115/115 [==============================] - 5s 42ms/step - loss: 0.6352 - accuracy: 0.6380 - val_loss: 0.6140 - val_accuracy: 0.6571
Epoch 5/50
115/115 [==============================] - 5s 41ms/step - loss: 0.6227 - accuracy: 0.6489 - val_loss: 0.6588 - val_accuracy: 0.6490
Epoch 6/50
115/115 [==============================] - 5s 42ms/step - loss: 0.6135 - accuracy: 0.6661 - val_loss: 0.6068 - val_accuracy: 0.6531
Epoch 7/50
115/115 [==============================] - 5s 40ms/step - loss: 0.5996 - accuracy: 0.6792 - val_loss: 0.6430 - val_accuracy: 0.6429
Epoch 8/50

In [24]:
# Emoticons
emoticons = \
	[	# For __EMOT_SMILEY
        (' __emoji: U+1F601',	[':-)', ':)', '(:', '(-:', ] )	,\
        # for __EMOT_LAUGH
		(' __emoji: U+1F923',		[':-D', ':D', 'X-D', 'XD', 'xD', ] )	,\
        # For __EMOT_LOVE
		(' __emoji: U+2764',		['<3', ':\*', ] )	,\
        # For __EMOT_WINK
		('__emoji: U+1F609',		[';-)', ';)', ';-D', ';D', '(;', '(-;', ] )	,\
        # For __EMOT_FROWN
		(' __emoji: U+2639',		[':-(', ':(', '(:', '(-:', ] )	,\
        # For __EMOT_CRY
		(' __emoji: U+1F622',		[':,(', ':\'(', ':"(', ':(('] )	,\
	]
    
def escape_paren(arr):
	return [text.replace(')', '[)}\]]').replace('(', '[({\[]') for text in arr]

def regex_union(arr):
	return '(' + '|'.join( arr ) + ')'

emoticons_regex = [ (repl, re.compile(regex_union(escape_paren(regx))) ) for (repl, regx) in emoticons ]

In [26]:
# Test
text = "This is a text with one emoticon :) and another :("
for (repl, regx) in emoticons_regex :
    text = re.sub(regx, ' '+repl+' ', text)
    
print(text)

This is a text with one emoticon   __emoji: U+1F601  and another   __emoji: U+2639 


In [28]:
emojis_db=pd.read_csv('training/emojis_db_csv.csv')
emojis_db.head()

,emoji_category,number,code,CLDR_Short_Name
0,face-positive,1,U+1F600,grinning face
1,face-positive,2,U+1F601,beaming face with smiling eyes
2,face-positive,3,U+1F602,face with tears of joy
3,face-positive,4,U+1F923,rolling on the floor laughing
4,face-positive,5,U+1F603,grinning face with big eyes


In [29]:
emojis_db.emoji_category.unique()

array(['face-positive', 'face-neutral', 'face-negative', 'face-sick',
       'face-role', 'face-fantasy', 'cat-face', 'monkey-face', 'person',
       'person-role', 'person-fantasy', 'person-gesture',
       'person-activity', 'person-sport', 'family', 'body', 'hair-style',
       'emotion', 'clothing', 'animal-mammal', 'animal-bird',
       'animal-amphibian', 'animal-reptile', 'animal-marine',
       'animal-bug', 'plant-flower', 'plant-other', 'food-fruit',
       'food-vegetable', 'food-prepared', 'food-asian', 'food-sweet',
       'drink', 'dishware', 'place-map', 'place-geographic',
       'place-building', 'place-religious', 'place-other',
       'transport-ground', 'transport-water', 'transport-air', 'hotel',
       'time', 'sky & weather', 'event', 'award-medal', 'sport', 'game',
       'arts & crafts', 'sound', 'music', 'musical-instrument', 'phone',
       'computer', 'light & video', 'book-paper', 'money', 'mail',
       'writing', 'office', 'lock', 'tool', 'science', 'medi

In [30]:
def emojis_unicode(tweet):
    ''' Extracts the emojis on the tweet on Unicode format, also tries to match those in regular format, such as ";)" '''
    for (repl, regx) in emoticons_regex :
        tweet = re.sub(regx, ' '+repl+' ', tweet)
#     print(tweet)

    tweet_unicode = str(tweet.encode('unicode-escape'))
    tweet_unicode = tweet_unicode.replace('\\\\U000',' __emoji: U+')
#     print(tweet_unicode)
    
    emoji_list = []
#     print(tweet)
    for emoji in range(tweet_unicode.count(' __emoji: ')):
        em = tweet_unicode.split('__emoji: ')[emoji+1].split()[0]
        em = em[:7] # the len of the emoji in unicode is between 6 and 7
        emoji_list.append(em.upper())
    return emoji_list

In [31]:
text = "This is a text with one emoticon :) and another :( and some others: 😡 🤣😂"
emojis_unicode(text)

['U+1F601', 'U+2639', 'U+1F621', 'U+1F923', 'U+1F602']

In [32]:
def emoji_category(emojis):
    categories = []
    for i in range(len(emojis)):
        # print(emojis[i])
        try:
            categories.append(emojis_db.loc[emojis_db.code == emojis[i]].emoji_category.values[0])
        except:
            try:
                _ = emojis[i].split('+')[1] + '+'
                categories.append(emojis_db[emojis_db.code.str.contains(_)].emoji_category.values[0])
            except:
                categories.append('other')
    if len(categories) < 1:
        categories.append('no_emojis')
    return categories

emoji_category(emojis_unicode(text))

['face-positive',
 'face-negative',
 'face-negative',
 'face-positive',
 'face-positive']

In [39]:
# Unigrams
unigrams_fd = nltk.FreqDist()
# unigrams_fd.update(text)
# unigrams_fd

# Bigrams
# words_bi  = [ ','.join(map(str,bg)) for bg in nltk.bigrams(text) ]
bi_grams_fd = nltk.FreqDist()
# bi_grams_fd.update( words_bi )
# bi_grams_fd

# Trigrams
# words_tri  = [ ','.join(map(str,tg)) for tg in nltk.trigrams(text) ]
tri_grams_fd = nltk.FreqDist()
# tri_grams_fd.update( words_tri )
# tri_grams_fd


In [40]:
# Wrapper function that encloses all the n-grams procedures
def get_word_features(words):
    bag = {}
    words_uni = [ 'has(%s)'% ug for ug in words ]
    words_bi  = [ 'has(%s)'% ','.join(map(str,bg)) for bg in nltk.bigrams(words) ]
    words_tri = [ 'has(%s)'% ','.join(map(str,tg)) for tg in nltk.trigrams(words) ]
    
    for f in words_uni+words_bi+words_tri:
        bag[f] = 1

    return bag

In [33]:
def extract_features(text):
    global usr_names, hstgs
    features = {}
    words = text
    words = processAll(text)

    word_features = get_word_features(words)
    features.update( word_features )

    negation_features = get_negation_features(words)
    features.update( negation_features )
    
    # Sentiment features are not included on the final deliverabe as did not improve results
#     sentiment_features = get_polarity_features(text)
#     features.update(sentiment_features )
    
    emoji_features = emoji_category(emojis_unicode(text))
    emoji_features_dic = dict( zip(['emoji_('+w+')' for w in  emoji_features], emoji_features))
    features.update( emoji_features_dic )
    
    usr_names = list(set(usr_names))
    hstgs = list(set(hstgs))
    return features

In [35]:
negtn_regex = re.compile( r"""(?:
    ^(?:nunca|no|nada|ningún|ninguno|ninguna|tampoco|
        nunc|nad|ningun|tampoc
    )$
)
|
n't
""", re.X)

def get_negation_features(words):
    INF = 0.0
    negtn = [ bool(negtn_regex.search(w)) for w in words ]

    left = [0.0] * len(words)
    prev = 0.0
    for i in range(0,len(words)):
        if( negtn[i] ):
            prev = 1.0
        left[i] = prev
        prev = max( 0.0, prev-0.1)

    right = [0.0] * len(words)
    prev = 0.0
    for i in reversed(range(0,len(words))):
        if( negtn[i] ):
            prev = 1.0
        right[i] = prev
        prev = max( 0.0, prev-0.1)

    return dict( zip(
                    ['neg_l('+w+')' for w in  words] + ['neg_r('+w+')' for w in  words],
                    left + right ) )

In [41]:
train['processed_tweet_features'] = train.tweet.apply(extract_features)

In [42]:
train.head()

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1,processed_tweet,processed_tweet_features
100001,100001,es,"@TheChiflis Ignora al otro, es un capullo.El p...",6,"[Annotator_1, Annotator_2, Annotator_3, Annota...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, YES, NO, YES, YES, YES]","[REPORTED, JUDGEMENTAL, -, REPORTED, JUDGEMENT...","[[OBJECTIFICATION], [OBJECTIFICATION, SEXUAL-V...",TRAIN_ES,YES,"[__user_THECHIFLIS, ignor, otro, capull, probl...","{'has(__user_THECHIFLIS)': 1, 'has(ignor)': 1,..."
100005,100005,es,@novadragon21 @icep4ck @TvDannyZ Entonces como...,6,"[Annotator_19, Annotator_20, Annotator_21, Ann...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, NO, YES, NO, YES, YES]","[REPORTED, -, JUDGEMENTAL, -, JUDGEMENTAL, DIR...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION], [-...",TRAIN_ES,YES,"[__user_NOVADRAGON21, __user_ICEP4CK, __user_T...","{'has(__user_NOVADRAGON21)': 1, 'has(__user_IC..."
100008,100008,es,@BestKabest Esta gringa sigue llorando por el ...,6,"[Annotator_25, Annotator_26, Annotator_27, Ann...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, DIRECT, JUDGEMENTAL, DIRECT, ...","[[IDEOLOGICAL-INEQUALITY], [STEREOTYPING-DOMIN...",TRAIN_ES,YES,"[__user_BESTKABEST, esta, gring, sig, llor, po...","{'has(__user_BESTKABEST)': 1, 'has(esta)': 1, ..."
100028,100028,es,"@ShahidForChange @TeamPelosi Quiet, sexist ^%$...",6,"[Annotator_109, Annotator_110, Annotator_111, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, NO, NO, YES, YES, YES]","[JUDGEMENTAL, -, -, DIRECT, DIRECT, DIRECT]","[[STEREOTYPING-DOMINANCE], [-], [-], [IDEOLOGI...",TRAIN_ES,YES,"[__user_SHAHIDFORCHANGE, __user_TEAMPELOSI, qu...","{'has(__user_SHAHIDFORCHANGE)': 1, 'has(__user..."
100036,100036,es,@Harassed_girl loca d mierda en k momento,6,"[Annotator_115, Annotator_116, Annotator_117, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, YES, YES, YES, YES, NO]","[DIRECT, DIRECT, DIRECT, DIRECT, DIRECT, -]","[[STEREOTYPING-DOMINANCE, SEXUAL-VIOLENCE], [M...",TRAIN_ES,YES,"[__user_HARASSED_GIRL, loc, mierd, moment]","{'has(__user_HARASSED_GIRL)': 1, 'has(loc)': 1..."


In [ ]:
train_size = int(train_.shape[0]*0.8)
sentiment_train_tweets = [(tweet, sentiment) for tweet, sentiment in train_[['tweetText', 'polarity_value']].values[:train_size]]
sentiment_train_tweets_full = [(tweet, sentiment) for tweet, sentiment in train_[['tweetText', 'polarity_value']].values]
sentiment_validation_tweets = [(tweet, sentiment) for tweet, sentiment in train_[['tweetText', 'polarity_value']].values[train_size:]]
sentiment_test_tweets  = [(tweet, sentiment) for tweet, sentiment in test_[['tweetText', 'polarity_value']].values]